In [8]:
import boto3
import datetime
import json
import zipfile

In [3]:
# create role lambda

trust_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "lambda.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}
iam_client = boto3.client('iam')
role_name = 'MyLambdaExecutionRole'
try:
    create_role_response = iam_client.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument=json.dumps(trust_policy),
        Description='Role for Lambda function with basic execution permissions'
    )
    role_arn = create_role_response['Role']['Arn']
    print(f"Created role with ARN: {role_arn}")
    # Attach AWSLambdaBasicExecutionRole policy
    iam_client.attach_role_policy(
        RoleName=role_name,
        PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
    )
    print(f"Attached AWSLambdaBasicExecutionRole policy to role {role_name}")
except iam_client.exceptions.EntityAlreadyExistsException:
    print(f"Role {role_name} already exists.")
    role_arn = iam_client.get_role(RoleName=role_name)['Role']['Arn']

Role MyLambdaExecutionRole already exists.


In [11]:
# create function
client = lambda_client = boto3.client('lambda')

ZIP_FILE = "hello_dunia.zip"
SOURCE_FILE = "hello_dunia.py"

with zipfile.ZipFile(ZIP_FILE, 'w', compression=zipfile.ZIP_DEFLATED) as z:
    # arcname='lambda_function.py' memastikan file ada di root zip, bukan di folder
    z.write(SOURCE_FILE, arcname=SOURCE_FILE) 

print("✅ Zip created successfully.")

function_name = 'HelloDuniaFunction'

try:
    create_function_response = client.create_function(
        FunctionName=function_name,
        Runtime='python3.11',
        Role=role_arn,
        Handler='hello_dunia.lambda_handler',
        Code={'ZipFile': open(ZIP_FILE, 'rb').read()},
        Description='A simple Lambda function that says Hello Dunia',
        Timeout=15,
        MemorySize=128,
        Publish=True,
    )
    print(f"Created Lambda function with ARN: {create_function_response['FunctionArn']}")
except client.exceptions.ResourceConflictException:
    print(f"Function {function_name} already exists.")
except Exception as e:
    print(f"Error creating function: {e}")

✅ Zip created successfully.
Function HelloDuniaFunction already exists.


In [12]:
# update lambda function

try:
    update_code_response = client.update_function_code(
        FunctionName=function_name,
        ZipFile=open(ZIP_FILE, 'rb').read(),
        Publish=True
    )
    print(f"Updated Lambda function code. New version: {update_code_response['Version']}")
except Exception as e:
    print(f"Error updating function code: {e}")

Updated Lambda function code. New version: 2


In [13]:
# invoke lambda function
# apa itu invoke disini?
# jawab: invoke itu memanggil fungsi lambda yang sudah dibuat tadi

client_lambda = boto3.client('lambda')
try:
    invoke_response = client_lambda.invoke(
        FunctionName=function_name,
        InvocationType='RequestResponse',
        LogType='Tail'
    )
    payload = invoke_response['Payload'].read().decode('utf-8')
    print(f"Lambda function response: {payload}")
except Exception as e:
    print(f"Error invoking Lambda function: {e}")

Lambda function response: {"statusCode": 200, "body": "Hello, Dunia! from AWS Lambda"}


# Create bucket

In [14]:
s3 = boto3.client('s3')

# create bucket
bucket_name = f"my-lambda-bucket-{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}"
try:
    create_bucket_response = s3.create_bucket(
        Bucket=bucket_name,
        CreateBucketConfiguration={
            'LocationConstraint': boto3.session.Session().region_name
        }
    )
    print(f"Created bucket: {bucket_name}")
except s3.exceptions.BucketAlreadyOwnedByYou:
    print(f"Bucket {bucket_name} already exists and is owned by you.")
except Exception as e:
    print(f"Error creating bucket: {e}")

Created bucket: my-lambda-bucket-20260210100140


In [38]:
# upload to S3
csv_path = "../data/raw/ecommerce/2024-01-15/orders.csv"
object_name = f"bronze/orders/{datetime.datetime.now().strftime('%Y-%m-%d')}/orders.csv"
try:
    s3.upload_file(
        Filename=csv_path,
        Bucket=bucket_name,
        Key=object_name
    )
    print(f"Uploaded {csv_path} to s3://{bucket_name}/orders/2024-01-15/orders.csv")
except Exception as e:
    print(f"Error uploading file to S3: {e}")

Uploaded ../data/raw/ecommerce/2024-01-15/orders.csv to s3://my-lambda-bucket-20260210100140/orders/2024-01-15/orders.csv


In [22]:
ZIP_FILE = "csv_validation.zip"
SOURCE_FILE = "csv_validation.py"

with zipfile.ZipFile(ZIP_FILE, 'w', compression=zipfile.ZIP_DEFLATED) as z:
    z.write(SOURCE_FILE, arcname=SOURCE_FILE) 

print("✅ Zip created successfully.")

try:
    create_function_response = client.create_function(
        FunctionName='CSVValidationFunction',
        Runtime='python3.11',
        Role=role_arn,
        Handler='csv_validation.lambda_handler',
        Code={'ZipFile': open(ZIP_FILE, 'rb').read()},
        Description='A Lambda function to validate CSV files in S3',
        Timeout=30,
        MemorySize=128,
        Publish=True,
    )
    print(f"Created Lambda function with ARN: {create_function_response['FunctionArn']}")
except client.exceptions.ResourceConflictException:
    print("Function CSVValidationFunction already exists.")
except Exception as e:
    print(f"Error creating function: {e}")

✅ Zip created successfully.
Function CSVValidationFunction already exists.


In [23]:
# UPDATE lambda function for CSV validation
try:
    update_code_response = client.update_function_code(
        FunctionName='CSVValidationFunction',
        ZipFile=open(ZIP_FILE, 'rb').read(),
        Publish=True
    )
    print(f"Updated Lambda function code. New version: {update_code_response['Version']}")
except Exception as e:
    print(f"Error updating function code: {e}")

Updated Lambda function code. New version: 3


In [39]:
# invoke

input_event = {
    "detail": {
        "bucket": {
            "name": bucket_name
        },
        "object": {
            "key": object_name
        }
    }
}

try:
    invoke_response = client_lambda.invoke(
            FunctionName='CSVValidationFunction',
            InvocationType='RequestResponse',
            LogType='Tail',
            Payload=json.dumps(input_event)
        )
    payload = invoke_response['Payload'].read().decode('utf-8')
    print(f"CSVValidationFunction response: {payload}")
except Exception as e:
    print(f"Error invoking CSVValidationFunction: {e}")

CSVValidationFunction response: {"status": "duplicate", "output": "silver/orders/2026-02-10/orders.parquet"}


In [32]:
!aws sqs create-queue --queue-name lambda-failures-dlq

{
    "QueueUrl": "https://sqs.ap-southeast-1.amazonaws.com/730335315755/lambda-failures-dlq"
}


In [34]:
!aws lambda update-function-configuration \
  --function-name CSVValidationFunction\
  --dead-letter-config TargetArn=arn:aws:sqs:ap-southeast-1:730335315755:lambda-failures-dlq

{
    "FunctionName": "CSVValidationFunction",
    "FunctionArn": "arn:aws:lambda:ap-southeast-1:730335315755:function:CSVValidationFunction",
    "Runtime": "python3.11",
    "Role": "arn:aws:iam::730335315755:role/MyLambdaExecutionRole",
    "Handler": "csv_validation.lambda_handler",
    "CodeSize": 843,
    "Description": "A Lambda function to validate CSV files in S3",
    "Timeout": 30,
    "MemorySize": 128,
    "LastModified": "2026-02-10T04:16:57.000+0000",
    "CodeSha256": "q+f/t1TBhzhwsfCkF8I80RsTpHdeWvq0YwKSAlZRYnw=",
    "Version": "$LATEST",
    "DeadLetterConfig": {
        "TargetArn": "arn:aws:sqs:ap-southeast-1:730335315755:lambda-failures-dlq"
    },
    "TracingConfig": {
        "Mode": "PassThrough"
    },
    "RevisionId": "cba8c723-3dd8-4f05-a219-09300504f965",
    "Layers": [
        {
            "Arn": "arn:aws:lambda:ap-southeast-1:336392948345:layer:AWSSDKPandas-Python311:25",
            "CodeSize": 55827444
        }
    ],
    "State": "Active",
    "La

In [40]:
# upload to S3 data jelek
csv_path = "../data/raw/ecommerce/2024-01-15/orders_jelek.csv"
object_name = f"bronze/orders/{datetime.datetime.now().strftime('%Y-%m-%d')}/orders_jelek.csv"
try:
    s3.upload_file(
        Filename=csv_path,
        Bucket=bucket_name,
        Key=object_name
    )
    print(f"Uploaded {csv_path} to {object_name}")
except Exception as e:
    print(f"Error uploading file to S3: {e}")

Uploaded ../data/raw/ecommerce/2024-01-15/orders_jelek.csv to bronze/orders/2026-02-10/orders_jelek.csv


In [ ]:
# invoke
input_event = {
    "detail": {
        "bucket": {
            "name": bucket_name
        },
        "object": {
            "key": object_name
        }
    }
}

try:
    invoke_response = client_lambda.invoke(
            FunctionName='CSVValidationFunction',
            InvocationType='Event',
            LogType='Tail',
            Payload=json.dumps(input_event)
        )
    print("Lambda dipanggil secara Asynchronous. Cek SQS dalam 1-2 menit.")
except Exception as e:
    print(f"Error invoking CSVValidationFunction: {e}")

Lambda dipanggil secara Asynchronous. Cek SQS dalam 1-2 menit.


In [ ]:
eventbridge_client = boto3.client('events')

# CREATE rule schedule
eventbridge_client.put_rule(
    Name='DailyCSVValidationRule',
    # rate 2 minutes
    ScheduleExpression='rate(2 minutes)',
    State='ENABLED',
    Description='Rule to trigger CSVValidationFunction every 2 minutes'
)

# ADD target to rule
lambda_arn = client.get_function(FunctionName='CSVValidationFunction')['Configuration']['FunctionArn']
eventbridge_client.put_targets(
    Rule='DailyCSVValidationRule',
    Targets=[
        {
            'Id': 'CSVValidationFunctionTarget',
            'Arn': lambda_arn
        }
    ]
)